In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks


In [3]:
import numpy as np
import json
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor

In [4]:
sonido_dic=np.load("datasetNuevo/sonidos.npy",allow_pickle=True).item()
print(len(sonido_dic))

9


In [5]:
partituras_dic=np.load("datasetNuevo/partituras.npy",allow_pickle=True).item()
print(partituras_dic)

{'9.xml': [1, 35, 35, 9, 9, 28, 28, 5, 5, 39, 39, 15, 15, 32, 32, 11, 11, 21, 21, 21, 21, 2], '8.xml': [1, 42, 42, 38, 38, 18, 18, 3, 3, 12, 12, 3, 3, 18, 18, 38, 38, 42, 42, 43, 43, 2], '2.xml': [1, 42, 42, 38, 38, 18, 18, 3, 3, 12, 12, 3, 3, 18, 18, 38, 38, 42, 42, 43, 43, 2], '7.xml': [1, 21, 21, 11, 11, 32, 32, 15, 15, 39, 39, 5, 5, 28, 28, 9, 9, 35, 35, 43, 43, 2], '1.xml': [1, 21, 21, 11, 11, 32, 32, 15, 15, 39, 39, 5, 5, 28, 28, 9, 9, 35, 35, 43, 43, 2], '4.xml': [1, 21, 21, 11, 11, 32, 32, 15, 15, 39, 39, 5, 5, 28, 28, 9, 9, 35, 35, 43, 43, 2], '3.xml': [1, 35, 35, 9, 9, 28, 28, 5, 5, 39, 39, 15, 15, 32, 32, 11, 11, 21, 21, 21, 21, 2], '6.xml': [1, 35, 35, 9, 9, 28, 28, 5, 5, 39, 39, 15, 15, 32, 32, 11, 11, 21, 21, 21, 21, 2], 'boqueron.xml': [1, 7, 20, 6, 20, 6, 24, 41, 33, 30, 29, 34, 16, 10, 8, 36, 29, 22, 25, 17, 7, 20, 6, 20, 6, 24, 41, 33, 30, 29, 34, 16, 10, 8, 36, 29, 22, 25, 17, 31, 31, 34, 22, 4, 14, 13, 29, 27, 29, 16, 26, 14, 13, 24, 37, 40, 23, 31, 31, 34, 22, 4, 1

In [6]:
with open("datasetNuevo/vocabulario2.json", "r") as fp:
        vocabPart = json.load(fp)

vocabPart

{'60 2.0 None': 21,
 '62 2.0 None': 11,
 '64 2.0 None': 32,
 '65 2.0 None': 15,
 '67 0.25 None': 7,
 '67 1.0 start': 40,
 '67 1.0 stop': 23,
 '67 2.0 None': 39,
 '69 0.25 None': 20,
 '69 0.5 None': 37,
 '69 2.0 None': 5,
 '71 0.25 None': 24,
 '71 0.25 start': 14,
 '71 0.25 stop': 13,
 '71 0.5 None': 6,
 '71 0.75 None': 4,
 '71 1.0 start': 25,
 '71 1.0 stop': 17,
 '71 2.0 None': 28,
 '72 2.0 None': 9,
 '74 0.5 None': 22,
 '74 2.0 None': 35,
 '75 0.25 None': 27,
 '75 0.5 None': 26,
 '76 0.25 None': 29,
 '76 0.25 start': 8,
 '76 0.25 stop': 36,
 '76 0.5 None': 34,
 '76 1.0 start': 33,
 '76 1.0 stop': 30,
 '76 2.0 None': 43,
 '77 2.0 None': 42,
 '78 0.25 None': 16,
 '79 0.5 None': 31,
 '79 0.75 None': 10,
 '79 1.0 None': 41,
 '79 2.0 None': 38,
 '81 2.0 None': 18,
 '83 2.0 None': 3,
 '84 2.0 None': 12,
 '<eos>': 2,
 '<pad>': 0,
 '<sos>': 1,
 'r 1.0 stop': 19}

In [7]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import  DataLoader
import numpy as np
def cargaDatos():
    data=[]
    sonido_dic=np.load("datasetNuevo/sonidos.npy",allow_pickle=True).item()
    partituras_dic=np.load("datasetNuevo/partituras.npy",allow_pickle=True).item()
    
    for sonido in sonido_dic:
        for part in partituras_dic:
            if part[:-4]==sonido[:-4]:
                audio=torch.tensor(sonido_dic[sonido],dtype=torch.float64)
                partitura=torch.tensor(partituras_dic[part])
                data.append((audio,partitura))
    
    return data

In [8]:
def generate_batch(data_batch):
    audio_batch, partitura_batch = [], []
    for (audio, partitura) in data_batch:
        audio_batch.append(torch.cat([torch.tensor([1]), audio, torch.tensor([2])], dim=0))
        partitura_batch.append( partitura)
    audio_batch = pad_sequence(audio_batch, padding_value=0)
    partitura_batch = pad_sequence(partitura_batch, padding_value=0)
    
    print(partitura_batch.shape,audio_batch.shape)
    return audio_batch, partitura_batch

In [9]:

BATCH_SIZE = 3
PAD_IDX = vocabPart['<pad>']
SOS_IDX = vocabPart['<sos>']
EOS_IDX = vocabPart['<eos>']

print(PAD_IDX)
print(SOS_IDX)
print(EOS_IDX)

0
1
2


In [10]:
muestra= cargaDatos()
train_dataset= DataLoader(muestra,batch_size=BATCH_SIZE,collate_fn=generate_batch,shuffle=True)
x ,y=next(iter(train_dataset))
print(x.shape)
print(x)
print(y.shape)
print(y)

torch.Size([22, 3]) torch.Size([9986, 3])
torch.Size([9986, 3])
tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00],
        [5.5741e+00, 5.5741e+00, 1.2859e+00],
        [1.6378e+00, 1.6378e+00, 1.1554e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 2.0090e-06],
        [0.0000e+00, 0.0000e+00, 9.3739e-06],
        [0.0000e+00, 0.0000e+00, 2.0000e+00]], dtype=torch.float64)
torch.Size([22, 3])
tensor([[ 1,  1,  1],
        [42, 42, 35],
        [42, 42, 35],
        [38, 38,  9],
        [38, 38,  9],
        [18, 18, 28],
        [18, 18, 28],
        [ 3,  3,  5],
        [ 3,  3,  5],
        [12, 12, 39],
        [12, 12, 39],
        [ 3,  3, 15],
        [ 3,  3, 15],
        [18, 18, 32],
        [18, 18, 32],
        [38, 38, 11],
        [38, 38, 11],
        [42, 42, 21],
        [42, 42, 21],
        [43, 43, 21],
        [43, 43, 21],
        [ 2,  2,  2]])


In [11]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
                
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)


In [12]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 10000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(20000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + 
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [17]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
tensorP =TokenEmbedding(vocab_size=len(vocabPart),emb_size=512)
tensorP

TokenEmbedding(
  (embedding): Embedding(44, 512)
)

In [14]:
prueba=tensorP(x)
prueba

IndexError: ignored

In [18]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [19]:
m= generate_square_subsequent_mask(42)
m.shape

torch.Size([42, 42])

In [20]:
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask=create_mask(x,y)

In [21]:
print(src_mask.shape)
print(src_mask)
print(tgt_mask.shape)
print(tgt_mask)



torch.Size([9986, 9986])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
torch.Size([22, 22])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -in

In [22]:
print(src_padding_mask.shape)
print(src_padding_mask)
print(tgt_padding_mask.shape)
print(tgt_padding_mask)

torch.Size([3, 9986])
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ..., False, False, False]])
torch.Size([3, 22])
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False]])


In [ ]:
SRC_VOCAB_SIZE = x.shape[0]
TGT_VOCAB_SIZE = len(vocabPart)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
NUM_EPOCHS = 16

In [ ]:


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

In [ ]:
print(transformer)

In [ ]:
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(transformer):,} parametros entrenables')

In [ ]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)
            
      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)
      
      optimizer.zero_grad()
      
      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)


def evaluate(model, val_iter):
  model.eval()
  losses = 0
  for idx, (src, tgt) in (enumerate(valid_iter)):
    src = src.to(device)
    tgt = tgt.to(device)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
    tgt_out = tgt[1:,:]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()
  return losses / len(val_iter)

In [ ]:
import io
import time
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
for epoch in range(1, NUM_EPOCHS+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_dataset, optimizer)
  end_time = time.time()
  
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: , "
          f"Epoch time = {(end_time - start_time):.3f}s"))

In [ ]:
import io
import time
for epoch in range(1, NUM_EPOCHS+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_iter, optimizer)
  end_time = time.time()
  val_loss = evaluate(transformer, valid_iter)
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s"))